In [1]:
from ikflow.model_loading import get_ik_solver
from ikflow.ikflow_solver import draw_latent
from ikflow.config import DEVICE
import torch
import numpy as np
from time import time, sleep
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.curdir, '..')))
from src.utils import RepoDir, BuildEnv, DrawAxes
import numpy as np
from pydrake.all import (
    StartMeshcat, 
    RigidTransform,
    Quaternion,
)


model_name = "panda__full__lp191_5.25m"

ik_solver, hyper_parameters = get_ik_solver(model_name)
robot = ik_solver.robot


meshcat = StartMeshcat()
diagram = BuildEnv(meshcat=meshcat, directives_file = os.path.join(RepoDir(), "models/panda/panda_collision.yaml"))
plant = diagram.GetSubsystemByName("plant")
diagram_context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(diagram_context)
diagram.ForcedPublish(diagram_context)

ikflow/config.py | Using device: 'cuda:0'
WorldModel::LoadRobot: /home/tangles/.cache/jrl/urdfs/panda_arm_hand_formatted_link_filepaths_absolute.urdf
joint mimic: no multiplier, using default value of 1 
joint mimic: no offset, using default value of 0 
URDFParser: Link size: 17
URDFParser: Joint size: 12
Geometry: Loading 12 meshes from /home/tangles/Urop/ikflow/.venv/lib/python3.10/site-packages/jrl/urdfs/panda/meshes/visual/link0.dae into Group
Geometry: Loading 4 meshes from /home/tangles/Urop/ikflow/.venv/lib/python3.10/site-packages/jrl/urdfs/panda/meshes/visual/link3.dae into Group
Geometry: Loading 4 meshes from /home/tangles/Urop/ikflow/.venv/lib/python3.10/site-packages/jrl/urdfs/panda/meshes/visual/link4.dae into Group
Geometry: Loading 3 meshes from /home/tangles/Urop/ikflow/.venv/lib/python3.10/site-packages/jrl/urdfs/panda/meshes/visual/link5.dae into Group
Geometry: Loading 17 meshes from /home/tangles/Urop/ikflow/.venv/lib/python3.10/site-packages/jrl/urdfs/panda/meshes

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [2]:
latent = torch.tensor(np.random.randn(1,ik_solver.network_width), device=DEVICE, dtype=torch.float32) 
T = RigidTransform(np.array([[0, 0, 1, -0.1],[1, 0, 0, 0],[0, 1, 0, 0],[0, 0, 0, 1]]))
target_pose = [0.3, 0.2, 0.5, 0.7071, 0, 0.7071, 0]  # x,y,z,qw,qx,qy,qz
target_pose = torch.tensor(target_pose, device=DEVICE)
conditional = torch.cat([target_pose.expand((1, ik_solver.network_width)), torch.zeros((1, 1), dtype=torch.float32, device=DEVICE)], dim=1)
sol = ik_solver.nn_model(latent, c=conditional, rev=True)

print(sol)

(tensor([[-3.0458,  0.7966, -2.7663, -2.6644, -2.0774,  2.7291,  0.0609]],
       device='cuda:0', grad_fn=<MmBackward0>), tensor([-28.6538], device='cuda:0', grad_fn=<AddBackward0>))


In [3]:
# Compute Jacobian dq/dz: derivatives of solutions w.r.t. latent
ik_solver.nn_model.eval()
latent_grad = torch.tensor(np.random.randn(1, ik_solver.network_width), device=DEVICE, dtype=torch.float32, requires_grad=True)

# Fix conditional: should be 7D pose + 1 zero softflow column = 8D
target_pose_fixed = torch.tensor(target_pose, device=DEVICE)
conditional_fixed = torch.cat([target_pose_fixed.unsqueeze(0), torch.zeros((1, 1), dtype=torch.float32, device=DEVICE)], dim=1)

# Define function mapping single latent sample to joint angles
def f_single(z):
    """Map latent vector z (1D) -> joint angles q (7D)"""
    z_batch = z.unsqueeze(0)  # (1, D)
    output, _ = ik_solver.nn_model(z_batch, c=conditional_fixed, rev=True)

    q = output[:, :7].squeeze(0)  # Extract first 7 dims (joint angles), remove batch dim
    return q

jacobian_gen = torch.func.jacrev(f_single)

/home/tangles/Urop/ikflow/.venv/lib/python3.10/site-packages/torch/utils/_device.py:78: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)


In [4]:
jacobian_single = jacobian_gen(latent_grad[0])
print(f"Jacobian shape (single sample): {jacobian_single.shape}")
print(f"Expected shape: (7, {ik_solver.network_width})")
print(f"\nJacobian (dq/dz):\n{jacobian_single}")

Jacobian shape (single sample): torch.Size([7, 7])
Expected shape: (7, 7)

Jacobian (dq/dz):
tensor([[-1.4473,  0.9082, -0.3038, -0.0138, -0.3171, -0.0536, -0.1551],
        [ 0.5631, -0.3632,  0.1150,  0.0046,  0.1259,  0.0225,  0.0607],
        [ 1.1077, -0.6893,  0.2384,  0.0086,  0.2448,  0.0376,  0.1154],
        [ 0.2976, -0.1771,  0.0623,  0.0019,  0.0657,  0.0084,  0.0293],
        [-0.7599,  0.4657, -0.1602, -0.0098, -0.1625, -0.0186, -0.0704],
        [ 0.9620, -0.6058,  0.2004,  0.0118,  0.2142,  0.0375,  0.1021],
        [ 1.5094, -0.9320,  0.3182,  0.0146,  0.3294,  0.0493,  0.1489]],
       device='cuda:0', grad_fn=<ViewBackward0>)


In [5]:
print(ik_solver.nn_model.training)

False
